In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



# Imports

In [ ]:
!pip install ultralytics

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image
import cv2
from ultralytics import YOLO
import torch
import warnings
warnings.filterwarnings('ignore')

In [ ]:
yaml_path = "/kaggle/input/balanced-yolo-data/data.yaml"

# Train Class

In [ ]:
from ultralytics import YOLO

class YOLOTrainer:
    def __init__(self, task='', mode='', model='', imgsz=512, data='', plots=True, momentum=None, patience=7,
                 device=None, epochs=None, batch=None, learning_rate=None, optimizer=None, weight_decay=None, name='', exist_ok=True):
        self.task = task
        self.mode = mode
        self.model = model
        self.imgsz = imgsz
        self.data = data
        self.device = device
        self.epochs = epochs
        self.batch = batch
        self.name = name
        self.exist_ok = exist_ok
        self.learning_rate = learning_rate
        self.optimizer = optimizer
        self.weight_decay = weight_decay
        self.plots = plots
        self.momentum = momentum
        self.patience = patience

    def load_model(self):
        model = YOLO(self.model)  # Load the YOLO model
        return model

    def train(self):
        # Train the model
        model = self.load_model()
        model.train(task=self.task, 
                    mode=self.mode, 
                    data=self.data, 
                    device=self.device,
                    epochs=self.epochs, 
                    batch=self.batch, 
                    imgsz=self.imgsz,
                    name=self.name, 
                    exist_ok=self.exist_ok,  
                    lr0=self.learning_rate,
                    optimizer=self.optimizer, 
                    weight_decay=self.weight_decay,
                    plots=self.plots,
                    momentum=self.momentum,
                    patience=self.patience)
        result = model.val()
        return result.results_dict.get('metrics/mAP50(B)', 0) 


# Finetune using Optuna

In [ ]:
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler

# Define the Optuna optimization objective
def objective(trial):
    # Suggest values for the hyperparameters
    batch_size = trial.suggest_int('batch_size', 16, 32)  # Integer value for batch size
    lr0 = trial.suggest_loguniform('lr0', 1e-4, 1e-1)
    momentum = trial.suggest_uniform('momentum', 0.8, 0.98)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)  # Log-uniform for weight decay

    # Initialize the YOLOTrainer with the suggested hyperparameters
    trainer = YOLOTrainer(
        task='detect',
        mode='train',
        model='yolo11m.pt',  # Pretrained weights
        data=yaml_path,  # Set your dataset config path
        epochs=2,  # Use fewer epochs for faster trials
        batch=batch_size,
        learning_rate=lr0,
        momentum=momentum,
        optimizer='Adam',  # Can change based on preference
        weight_decay=weight_decay,
        name=f'optuna_trial_{trial.number}',  # Unique name for each trial
        device='cuda'  # Assuming you're using a GPU, change as needed
    )

    # Train and evaluate
    val_score = trainer.train()  # Replace with your validation metric
    trial.report(val_score, step=1)  # Report intermediate results to Optuna

    # Prune unpromising trials
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    return val_score  # Return the evaluation metric to Optuna for optimization


# Create an Optuna study and start optimization
study = optuna.create_study(
    direction='maximize',  # 'maximize' for higher scores (mAP, etc.)
    sampler=TPESampler(seed=42),  # Use TPE for efficiency
    pruner=MedianPruner(n_startup_trials=5)  # Prune after 5 trials
)

study.optimize(objective, n_trials=40, n_jobs=4)  # Parallelize with 4 threads (adjust as needed)

# Print the best trial found by Optuna
print(f"Best trial parameters: {study.best_trial.params}")
print(f"Best trial value: {study.best_trial.value}")


# Save Best Parameters

In [ ]:
import json

with open("best_params.json", "w") as f:
    json.dump(study.best_trial.params, f)
